In [ ]:
!pip install ultralytics
!git clone https://github.com/advadj67/hw3_M11221004.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.4/756.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
'''
#將.xml轉成Yolo所需 .txt檔案
import os
import xml.etree.ElementTree as ET

def convert_coordinates(size, box):
    # 轉換坐標
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders):
    # 解析XML並轉換為YOLO格式
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for output_folder in output_folders:
        folder_name = os.path.basename(output_folder)
        output_dir = os.path.join(output_root, folder_name)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_file_path = os.path.join(output_dir, os.path.splitext(os.path.basename(xml_path))[0] + '.txt')
        with open(output_file_path, 'w') as f:
            for obj in root.findall('object'):
                cls = obj.find('name').text
                if cls not in class_dict:
                    continue
                cls_id = class_dict[cls]
                xml_box = obj.find('bndbox')
                box = (float(xml_box.find('xmin').text), float(xml_box.find('xmax').text),
                       float(xml_box.find('ymin').text), float(xml_box.find('ymax').text))
                bb = convert_coordinates((w,h), box)
                f.write(f"{cls_id} {' '.join([str(a) for a in bb])}\n")

# 資料夾列表
folders = ["訓練集_xml", "驗證集_xml", "測試集_xml"]
o_folders = ["train", "val", "test"]

# 分類字典，將類別名映射到整數標籤
class_dict = {"container": 0}

# 輸出根資料夾路徑
output_root = "E:\Downloads\貨櫃資料集\labels"

# 如果輸出根資料夾不存在，則創建
if not os.path.exists(output_root):
    os.makedirs(output_root)

# 迴圈處理每個資料夾
for folder, o_folder in zip(folders, o_folders):
    folder_path = os.path.join("E:\Downloads\貨櫃資料集", folder)  # 資料夾路徑
    xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]  # 獲取所有XML檔案
    output_folders = [os.path.join(output_root, o_folder)]  # 輸出資料夾路徑
    for xml_file in xml_files:
        xml_path = os.path.join(folder_path, xml_file)  # XML檔案路徑
        convert_xml_to_yolo(xml_path, output_root, class_dict, output_folders)

print("labels轉換完成!")
'''

In [ ]:
'''
from PIL import Image

# 資料夾列表
folder_path = "E:/Downloads/貨櫃資料集/"
folders = [folder_path + "訓練集", folder_path + "驗證集", folder_path + "測試集"]

output_folders = [folder_path + "images/train", folder_path + "images/val", folder_path + "images/test"]


# 創建輸出資料夾
for output_folder in output_folders:
    os.makedirs(output_folder, exist_ok=True)

# 轉換函數
def resize_images(folder, output_folder):
    # 獲取資料夾中所有圖片的檔案名稱
    files = os.listdir(folder)

    # 迴圈處理每張圖片
    for file in files:
        # 檔案路徑
        file_path = os.path.join(folder, file)

        # 如果是檔案
        if os.path.isfile(file_path):
            # 打開圖片
            img = Image.open(file_path)

            # 重新調整大小
            resized_img = img.resize((416, 416))

            # 另存新圖片
            resized_img.save(os.path.join(output_folder, file))

# 對每個資料夾和對應的輸出資料夾調用resize_images函數
for folder, output_folder in zip(folders, output_folders):
    resize_images(folder, output_folder)

print("images轉換完成!")
'''

images轉換完成!


In [ ]:
#修改bounding box座標

def resize_bounding_boxes(label_file_path, original_size, new_size):
    # 讀取原始標籤文件
    with open(label_file_path, 'r') as f:
        lines = f.readlines()

    # 計算大小比例
    width_ratio = new_size[0] / original_size[0]
    height_ratio = new_size[1] / original_size[1]

    # 對每個 bounding box 進行調整
    for i, line in enumerate(lines):
        parts = line.strip().split()
        # 解析原始 bounding box 坐標
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        # 調整 bounding box 坐標
        x_center *= width_ratio
        y_center *= height_ratio
        width *= width_ratio
        height *= height_ratio
        # 更新標籤文件中的 bounding box 坐標
        lines[i] = f"{parts[0]} {x_center} {y_center} {width} {height}\n"

    # 將更新後的內容寫回標籤文件
    with open(label_file_path, 'w') as f:
        f.writelines(lines)

# 設置原始影像大小和新影像大小
original_size = (1920, 1080)
new_size = (416, 416)

# 設置資料集目錄路徑
dataset_dir = "/content/hw3_M11221004/貨櫃資料集/labels"
subfolders = ['train', 'val', 'test']

# 迴圈遍歷每個子資料夾中的標籤文件
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_dir, subfolder)
    label_files = [f for f in os.listdir(subfolder_path) if f.endswith('.txt')]
    for label_file in label_files:
        label_file_path = os.path.join(subfolder_path, label_file)
        # 調整 bounding box 坐標
        resize_bounding_boxes(label_file_path, original_size, new_size)


In [ ]:
#模型建立
import time
from ultralytics import YOLO

if __name__=='__main__':
    model = YOLO("yolov8n.pt")
    print("開始訓練 .........")
    t1 = time.time()
    results = model.train(data="/content/hw3_M11221004/貨櫃資料集/data.yaml", epochs=1)
    t2 = time.time()
    print(f'訓練花費時間 : {t2-t1}秒')

    # 儲存最佳模型
    best_model_path = "/content/best_model.pt"
    model.save(best_model_path)

    # 進行驗證
    val_results = model.val(data="/content/hw3_M11221004/貨櫃資料集/data.yaml")


開始訓練 .........
Ultralytics YOLOv8.2.13 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/hw3_M11221004/貨櫃資料集/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

train: Scanning /content/hw3_M11221004/貨櫃資料集/labels/train... 2125 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2125/2125 [00:01<00:00, 1183.18it/s]


train: New cache created: /content/hw3_M11221004/貨櫃資料集/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/hw3_M11221004/貨櫃資料集/labels/val... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<00:00, 668.98it/s]

val: New cache created: /content/hw3_M11221004/貨櫃資料集/labels/val.cache


...

Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 9.7ms postprocess per image
Results saved to runs/detect/train12
訓練花費時間 : 148.03404831886292秒
Ultralytics YOLOv8.2.13 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/hw3_M11221004/貨櫃資料集/labels/val.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:11<00:00,  2.96it/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 36008 (\N{CJK UNIFIED IDEOGRAPH-8CA8}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 27331 (\N{CJK UNIFIED IDEOGRAPH-6AC3}) missing from current font.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:465: UserWarning: Glyph 34399 (\N{

                   all        536        536      0.192      0.274     0.0803     0.0219


/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 36008 (\N{CJK UNIFIED IDEOGRAPH-8CA8}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 27331 (\N{CJK UNIFIED IDEOGRAPH-6AC3}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 34399 (\N{CJK UNIFIED IDEOGRAPH-865F}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 30908 (\N{CJK UNIFIED IDEOGRAPH-78BC}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 36008 (\N{CJK UNIFIED IDEOGRAPH-8CA8}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.10/dist

Speed: 0.5ms preprocess, 5.2ms inference, 0.0ms loss, 7.1ms postprocess per image
Results saved to runs/detect/train122


In [ ]:
if __name__=='__main__':
    best_model_path = "/content/best_model.pt"
    model = YOLO(best_model_path)

    # 進行預測
    results = model.predict(source="/content/hw3_M11221004/貨櫃資料集/測試集/image_0001.jpg", save=True)

    # 顯示預測結果
    results[0].show()



image 1/1 /content/hw3_M11221004/貨櫃資料集/測試集/image_0001.jpg: 384x640 1 truck, 25.8ms
Speed: 3.1ms preprocess, 25.8ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2


In [ ]:
!pip install easyocr

In [ ]:
import os  # 導入操作系統相關的庫
import platform  # 導入獲取系統信息的庫
import pylab as plt  # 導入繪圖庫
import cv2  # 導入 OpenCV 库
import numpy as np  # 導入處理數組的庫
import easyocr  # 導入 EasyOCR 库
from PIL import Image, ImageFont, ImageDraw  # 導入圖像處理庫中的相關模組
from ultralytics import YOLO  # 導入 YOLO 相關庫
import random

def text(img, text, xy=(0, 0), color=(0, 0, 0), size=20, font=None):
    pil = Image.fromarray(img)  # 將圖像轉換為 PIL.Image 格式
    if font is None:  # 如果未指定字體，則隨機選擇一個預先安裝的字體
        font_list = ['Arial', 'Times New Roman', 'Courier New']  # 預先安裝的字體列表
        font = random.choice(font_list)  # 從列表中隨機選擇一個字體
    font = ImageFont.truetype(font + '.ttf', size)  # 使用指定的字體和大小
    ImageDraw.Draw(pil).text(xy, text, font=font, fill=color)  # 在圖像上繪製文字
    return np.asarray(pil)  # 將 PIL.Image 格式轉換為 numpy.ndarray 格式

# 初始化 YOLO 模型，載入預訓練權重
model = YOLO('/content/best_model.pt')

# 初始化 EasyOCR 讀取器，指定支持的語言
reader = easyocr.Reader(['en'])

# 指定要處理的圖像路徑
path = "/content/hw3_M11221004/貨櫃資料集/images/test"

# 設置圖像顯示的大小
#plt.figure(figsize=(12, 9))

# 遍歷指定路徑下的前六張圖像
for i, file in enumerate(os.listdir(path)[0:6]):
    full = os.path.join(path, file)  # 獲取完整的圖像路徑
    print(full)  # 輸出圖像的完整路徑

    # 讀取圖像
    img = cv2.imdecode(np.fromfile(full, dtype=np.uint8), cv2.IMREAD_COLOR)
    img = img[:, :, ::-1].copy()  # 將圖像從 BGR 格式轉換為 RGB 格式

    # 使用 YOLO 模型進行物件偵測
    results = model.predict(img, save=False)
    boxes = results[0].boxes.xyxy  # 獲取偵測到的物件框的坐標

    # 遍歷每個物件框
    for box in boxes:
        x1 = int(box[0])  # 物件框左上角 x 座標
        y1 = int(box[1])  # 物件框左上角 y 座標
        x2 = int(box[2])  # 物件框右下角 x 座標
        y2 = int(box[3])  # 物件框右下角 y 座標

        # 在圖像上繪製物件框
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 從物件框中提取汽車區域，並使用 EasyOCR 库進行文字識別
        cropped_image = img[y1:y2, x1:x2]
        result = reader.readtext(cropped_image)

        # 如果識別結果不為空，則在圖像上繪製識別結果
        if result:
            text_to_write = result[0][1]  # 獲取識別結果中的第一個文字
            img = text(img, text_to_write, (x1, y1 - 20), (0, 255, 0), 25)

    # 將處理後的圖像顯示在指定的子圖中
    plt.subplot(2, 3, i + 1)
    plt.axis("off")  # 不顯示坐標軸
    plt.imshow(img)  # 顯示圖像

# 將處理後的圖像保存為文件
plt.savefig("yolov8_car.jpg")

# 顯示圖像
plt.show()

### **缺乏：**


1. mAP、Recall、Precision和F1-score
2. 準確率績效公式: 辨識正確筆數/測試資料筆數
3. 車牌偵測及文字辨識
4. 影片的






